In [1]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO, A2C
import librosa 
from pydub import AudioSegment
import pydub  
import pickle  

In [14]:
df = pd.read_csv('../data/simulation_training_hrv_recommender.csv')
df.head()

,tenderness,calmness,power,joyful_activation,tension,sadness,classical,electronic,pop,rock,...,age_young_adult,ratings,MEAN_RR,SDRR,HR,RMSSD,pNN50,VLF,LF,HF
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,955.393597,145.513407,64.661410,22.833293,1.25,2727.188512,2100.772584,6.942776
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,495.137333,20.541448,121.380129,4.649531,0.00,122.992404,172.812347,9.815828
2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1053.412598,89.129914,57.364457,26.631007,4.75,2025.397417,2910.018052,2.651670
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,684.916597,55.148493,88.146335,5.229125,0.00,729.405685,121.157088,4.408449
4,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,861.173850,75.949986,70.217896,15.026389,0.00,1795.627285,702.762118,32.335595


In [3]:
train.columns

Index([' tenderness', ' calmness', ' power', ' joyful_activation', ' tension',
       ' sadness', 'classical', 'electronic', 'pop', 'rock', ' gender',
       ' mood', ' age_adult', ' age_child', ' age_senior', ' age_young',
       ' age_young_adult', 'ratings', 'MEAN_RR', 'SDRR', 'HR', 'RMSSD',
       'pNN50', 'VLF', 'LF', 'HF'],
      dtype='object')

In [15]:
FEATURES = [' tenderness', ' calmness', ' power', ' joyful_activation', ' tension', ' sadness', 'classical', 'electronic',
                          'pop', 'rock', ' gender', ' mood',' age_adult', ' age_child', ' age_senior', ' age_young', ' age_young_adult']
HRV_FEATURES = ['MEAN_RR','SDRR','HR','RMSSD','pNN50','VLF','LF','HF']

In [23]:
class SimulationEnvironment(gym.Env):
    """Music Recommender system  Environment Simulation for gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, train, hrv_interpreter='../model_saved/hrv_classifier_model.sav'):
        super( SimulationEnvironment, self).__init__()
        # shuffle the date
        self.data =  train.copy().sample(frac=1, random_state=47)
        # set Actions space : Propose the Music or Not 
        self.action_space = spaces.Box(
            low=0, high=1
            , shape=(1,), dtype=np.float16)
        # Set observation space
        self.observation_space = spaces.Box(
            low=0, high=1
            , shape=(len(FEATURES),), dtype=np.float16)
        
        self.hrv_interpreter = pickle.load(open(hrv_interpreter, 'rb'))
        # Initial step 
        self.current_step = 0
        self.sum_reward = 0 
    def get_next_observation(self):
        obs = self.data[FEATURES].iloc[self.current_step].values
        print(obs)
        return obs
    def reset(self):
        self.data =  self.data.copy().sample(frac=1, random_state=47)
        self.current_step = 0
        self.sum_reward = 0 
        self.accuracy_score = 0 
        return self.get_next_observation()
    def step(self, action):
        # Retrieve user hrv 
        user_hrv = self.data[HRV_FEATURES].iloc[self.current_step].to_frame().T    
        # Interpret HRV
        user_stress = self.hrv_interpreter.predict(user_hrv)[0]
        # Add Some reward (+1) if the model propose  song that user likes 
        if user_stress == 0 and action[0] > 0.5: 
            reward = 1  * action[0]  
        elif user_stress !=0 and action[0] < 0.5: 
            reward = 1  * (1 - action[0])  
        else:
            reward = 0  #- 1 *  action[0] 
        
        
        if self.current_step > self.data.shape[0] - 10:
            done = True 
        else:
            self.current_step += 1 
            done = False 
        obs = self.get_next_observation()
        self.sum_reward += reward 
        self.mean_reward = self.sum_reward /  self.current_step 
        return obs, reward, done, {}
    def render(self, mode='human', close=False):
        # Render the environment to the screen
        if self.current_step % 10 == 0:
            print(f'Sum Observation:{self.current_step}, Sum reward:{self.sum_reward}, Success Rate:{self.sum_reward / self.current_step}')
           

In [24]:
#  Split Train and Validation 
split = 0.8
train = df.iloc[: int(df.shape[0] * 0.8), :].reset_index(drop=True)
val = df.iloc[int(df.shape[0] * 0.8):, :].reset_index(drop=True)

In [25]:
# Train the Model
train_env =  SimulationEnvironment(train)
model = A2C('MlpPolicy', train_env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env in a DummyVecEnv.
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 4. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 2. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 0. 1. 0.]
[0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 5. 0. 0. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 1. 0. 0. 0

[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 4. 0. 0. 1. 0. 0.]
[0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 5. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 

[0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 5. 1. 0. 0. 0. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 4. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 5. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 4. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 

[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 5. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 4. 0. 0. 0. 

[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 1. 0. 0.]
[0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 5. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 5. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 3. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 5. 0. 0. 0. 

...

[1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 1. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 5. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 4. 0. 0. 1. 

[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 2. 0. 0. 0. 0. 1.]
[0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 5. 0. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 4. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 5. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 4. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 4. 1. 0. 0. 

[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 3. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 3. 0. 0. 1. 0. 0.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 2. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 2. 0. 0. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 

[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 3. 1. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 5. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 1. 0. 0.]
[1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 1. 0. 0. 0. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 4. 0. 0. 0. 

[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 4. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 4. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 3. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 4. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 3. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 5. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 1.]
[0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 3. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 5. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 5. 0. 0. 0. 

In [22]:
#Save the model
model.save('recommender_hrv_based')

In [20]:
# Evaluate the model
val_env = SimulationEnvironment(val)
obs = val_env.reset()
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = val_env.step(action)
    val_env.render()
    if done:
        obs = val_env.reset()

Sum Observation:10, Sum reward:10.0, Success Rate:1.0
Sum Observation:20, Sum reward:20.0, Success Rate:1.0
Sum Observation:30, Sum reward:30.0, Success Rate:1.0
Sum Observation:40, Sum reward:40.0, Success Rate:1.0
Sum Observation:50, Sum reward:50.0, Success Rate:1.0
Sum Observation:60, Sum reward:60.0, Success Rate:1.0
Sum Observation:70, Sum reward:70.0, Success Rate:1.0
Sum Observation:80, Sum reward:80.0, Success Rate:1.0
Sum Observation:90, Sum reward:90.0, Success Rate:1.0
Sum Observation:100, Sum reward:100.0, Success Rate:1.0
